<a href="https://www.kaggle.com/code/sreejitamukherjee/mastercard-data-financial-modelling?scriptVersionId=137321751" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nifty-50-returns/Nifty_50_returns.xlsx
/kaggle/input/nifty50-last-15-years/Nifty_50_returns.xlsx
/kaggle/input/mastercard-stock-data-latest-and-updated/Mastercard_stock_info.csv
/kaggle/input/mastercard-stock-data-latest-and-updated/Mastercard_stock_history.csv
/kaggle/input/nifty-50-returns1/Nifty_50_returns.xlsx
/kaggle/input/returns-index/Nifty 50 returns.xlsx


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.seasonal import seasonal_decompose


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data= pd.read_csv('../input/mastercard-stock-data-latest-and-updated/Mastercard_stock_history.csv')

In [4]:
data1= pd.read_excel('/kaggle/input/nifty-50-returns1/Nifty_50_returns.xlsx')

In [5]:
data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2006-05-25,3.748967,4.283869,3.739664,4.279217,395343000,0.0,0.0
1,2006-05-26,4.307126,4.348058,4.103398,4.179680,103044000,0.0,0.0
2,2006-05-30,4.183400,4.184330,3.986184,4.093164,49898000,0.0,0.0
3,2006-05-31,4.125723,4.219679,4.125723,4.180608,30002000,0.0,0.0
4,2006-06-01,4.179678,4.474572,4.176887,4.419686,62344000,0.0,0.0


In [6]:
data=data.drop(['Dividends'],axis=1)
data=data.drop(['Stock Splits'],axis=1)

In [7]:
data.head()

,Date,Open,High,Low,Close,Volume
0,2006-05-25,3.748967,4.283869,3.739664,4.279217,395343000
1,2006-05-26,4.307126,4.348058,4.103398,4.179680,103044000
2,2006-05-30,4.183400,4.184330,3.986184,4.093164,49898000
3,2006-05-31,4.125723,4.219679,4.125723,4.180608,30002000
4,2006-06-01,4.179678,4.474572,4.176887,4.419686,62344000


In [8]:
data1.head()

,Date,Price,Open,High,Low,Volume,Chg%
0,2008-04-01,4739.55,4735.65,4800.75,4628.75,119.88B,0.0011
1,2009-04-01,3060.35,3023.85,3069.30,2965.70,319.35B,0.0130
2,2010-04-01,5290.50,5249.20,5298.60,5249.20,127.77B,0.0079
3,2011-04-01,5826.05,5835.00,5860.20,5810.40,118.32B,-0.0013
4,2013-04-01,5704.40,5697.35,5720.95,5675.90,97.78B,0.0038


# **Some basic data **
# Equity of risk premium 2023 (India)=9.73%

# **Risk free rate=7.07%**



In [9]:
adj_close = data['Close']

In [10]:
# Calculate the returns based on the percentage change of 'Adj Close' column
returns = adj_close.pct_change().rename('Returns').to_frame()
print(returns)

       Returns
0          NaN
1    -0.023261
2    -0.020699
3     0.021363
4     0.057187
...        ...
3867 -0.002876
3868  0.015967
3869  0.016253
3870  0.002967
3871 -0.022002

[3872 rows x 1 columns]


In [11]:
# Concatenate the adjusted close prices to the existing data DataFrame
data = pd.concat([data, adj_close.rename('Adj Close')], axis=1)

In [12]:
# Add the returns column to the DataFrame
data = pd.concat([data, returns], axis=1)

In [13]:
data.head()

,Date,Open,High,Low,Close,Volume,Adj Close,Returns
0,2006-05-25,3.748967,4.283869,3.739664,4.279217,395343000,4.279217,NaN
1,2006-05-26,4.307126,4.348058,4.103398,4.179680,103044000,4.179680,-0.023261
2,2006-05-30,4.183400,4.184330,3.986184,4.093164,49898000,4.093164,-0.020699
3,2006-05-31,4.125723,4.219679,4.125723,4.180608,30002000,4.180608,0.021363
4,2006-06-01,4.179678,4.474572,4.176887,4.419686,62344000,4.419686,0.057187


In [14]:
data=data.dropna()

In [15]:
data.head(1000)

,Date,Open,High,Low,Close,Volume,Adj Close,Returns
1,2006-05-26,4.307126,4.348058,4.103398,4.179680,103044000,4.179680,-0.023261
2,2006-05-30,4.183400,4.184330,3.986184,4.093164,49898000,4.093164,-0.020699
3,2006-05-31,4.125723,4.219679,4.125723,4.180608,30002000,4.180608,0.021363
4,2006-06-01,4.179678,4.474572,4.176887,4.419686,62344000,4.419686,0.057187
5,2006-06-02,4.511782,4.530387,4.352707,4.371312,37253000,4.371312,-0.010945
...,...,...,...,...,...,...,...,...
996,2010-05-11,21.432253,21.439794,20.850762,21.039251,29271000,21.039251,-0.030698
997,2010-05-12,21.205125,22.262554,21.166484,21.923271,29123000,21.923271,0.042018
998,2010-05-13,21.930813,22.611262,21.729128,21.894058,22049000,21.894058,-0.001333
999,2010-05-14,20.290948,20.735785,19.703801,20.022350,101983000,20.022350,-0.085489


In [16]:
data1['Date'] = data1['Date'].astype(str)

In [17]:
result = pd.merge(data, data1, on='Date', how='left')

In [18]:
result.head(1000)

,Date,Open_x,High_x,Low_x,Close,Volume_x,Adj Close,Returns,Price,Open_y,High_y,Low_y,Volume_y,Chg%
0,2006-05-26,4.307126,4.348058,4.103398,4.179680,103044000,4.179680,-0.023261,3209.60,3177.70,3277.95,3177.70,105.52B,0.0100
1,2006-05-26,4.307126,4.348058,4.103398,4.179680,103044000,4.179680,-0.023261,3209.60,3177.70,3277.95,3177.70,105.52B,0.0100
2,2006-05-30,4.183400,4.184330,3.986184,4.093164,49898000,4.093164,-0.020699,3185.30,3215.50,3252.10,3167.05,69.35B,-0.0092
3,2006-05-30,4.183400,4.184330,3.986184,4.093164,49898000,4.093164,-0.020699,3185.30,3215.50,3252.10,3167.05,69.35B,-0.0092
4,2006-05-31,4.125723,4.219679,4.125723,4.180608,30002000,4.180608,0.021363,3071.05,3181.95,3181.95,2972.90,119.37B,-0.0359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2008-07-31,23.118947,24.008317,22.255816,22.880907,160568000,22.880907,-0.098179,4332.95,4314.35,4342.00,4285.55,199.68B,0.0045
996,2008-08-01,22.536026,22.740327,21.132151,22.236132,111075000,22.236132,-0.028180,4413.55,4331.60,4422.95,4235.70,178.08B,0.0186
997,2008-08-04,21.835025,22.491979,21.472343,21.641031,41515000,21.641031,-0.026763,4395.35,4426.10,4436.15,4362.90,131.34B,-0.0041
998,2008-08-05,21.818158,22.221140,21.601672,22.221140,37083000,22.221140,0.026806,4502.85,4395.80,4515.15,4376.00,193.93B,0.0245


In [19]:
result = result.drop_duplicates(subset=['Date'])


In [20]:
result.head(1000)

,Date,Open_x,High_x,Low_x,Close,Volume_x,Adj Close,Returns,Price,Open_y,High_y,Low_y,Volume_y,Chg%
0,2006-05-26,4.307126,4.348058,4.103398,4.179680,103044000,4.179680,-0.023261,3209.60,3177.70,3277.95,3177.70,105.52B,0.0100
2,2006-05-30,4.183400,4.184330,3.986184,4.093164,49898000,4.093164,-0.020699,3185.30,3215.50,3252.10,3167.05,69.35B,-0.0092
4,2006-05-31,4.125723,4.219679,4.125723,4.180608,30002000,4.180608,0.021363,3071.05,3181.95,3181.95,2972.90,119.37B,-0.0359
6,2006-06-01,4.179678,4.474572,4.176887,4.419686,62344000,4.419686,0.057187,2962.25,3072.55,3130.00,2936.60,112.88B,-0.0354
8,2006-06-02,4.511782,4.530387,4.352707,4.371312,37253000,4.371312,-0.010945,3091.35,2961.50,3099.35,2940.20,106.18B,0.0436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442,2010-05-11,21.432253,21.439794,20.850762,21.039251,29271000,21.039251,-0.030698,5136.15,5189.75,5206.70,5126.50,168.75B,-0.0111
1443,2010-05-12,21.205125,22.262554,21.166484,21.923271,29123000,21.923271,0.042018,5156.65,5133.75,5172.85,5098.80,187.11B,0.0040
1444,2010-05-13,21.930813,22.611262,21.729128,21.894058,22049000,21.894058,-0.001333,5178.90,5157.55,5212.70,5147.95,179.38B,0.0043
1445,2010-05-14,20.290948,20.735785,19.703801,20.022350,101983000,20.022350,-0.085489,5093.50,5180.55,5192.75,5070.95,144.38B,-0.0165


In [21]:
# Assuming you have the returns of the stock and index stored in separate variables
stock_returns = result['Returns']
index_returns= result['Chg%']

# Calculate the covariance
covariance = np.cov(stock_returns, index_returns)[0,0]

print("Covariance:", covariance)


Covariance: 0.0004709314470417931


In [22]:
# Calculate the variance of the benchmark index returns
variance = np.var(index_returns)

# Calculate the beta coefficient
beta = covariance / variance

# Print the beta coefficient
print("Beta coefficient:", beta)

Beta coefficient: 2.3285056294949924


# A beta coefficient of 2.32 indicates that the asset is expected to be more volatile than the overall market. 

# **Specifically, it suggests that the asset's price is likely to fluctuate approximately 2.32 times as much as the market index.**



# Calculating cost of Equity

In [23]:
Equity_of_risk_premium=9.73

Risk_free_rate=7.07


In [24]:
coe=(beta*Equity_of_risk_premium)+Risk_free_rate
print(coe)

29.72635977498628


# **Thus it signifies that the company or investment is expected to generate a return of at least 29.73% in order to adequately compensate shareholders for the risk they are taking by investing in the company's equity**

Pretax Income	11,732	10,307	7,760	9,731	7204

Income Tax	1,802	1,620	1,349	1,613	1,3457204

In [25]:
import pandas as pd

# Create the data frames
tax_file = {
    'Dates': ['2022', '2021', '2020', '2019', '2018'],
    'Pretax Income': [11732, 10307, 7760, 9731, 7204],
    'Income Tax': [1802, 1620, 1349, 1613, 1345]
}

df = pd.DataFrame(tax_file)


# Set the 'Dates' column as the index
df.set_index('Dates', inplace=True)

# Print the data frames
print(df)


       Pretax Income  Income Tax
Dates                           
2022           11732        1802
2021           10307        1620
2020            7760        1349
2019            9731        1613
2018            7204        1345


In [26]:
#Calculate tax schedules

(df['Income Tax']/df['Pretax Income'])*100

Dates
2022    15.359700
2021    15.717474
2020    17.384021
2019    16.575891
2018    18.670183
dtype: float64

190	100	26	(42)	101

In [27]:
import pandas as pd

# Create the data frames
interest = {
    'Dates': ['2022', '2021', '2020', '2019', '2018'],
    'Interest Paid': [190,100,26,42,101],
    'Total Current Liabilities':[14171,13162,11847,11904,11593]
}

df1 = pd.DataFrame(interest)


# Set the 'Dates' column as the index
df1.set_index('Dates', inplace=True)

# Print the data frames
print(df1)


       Interest Paid  Total Current Liabilities
Dates                                          
2022             190                      14171
2021             100                      13162
2020              26                      11847
2019              42                      11904
2018             101                      11593


In [28]:
(df1['Interest Paid']/df1['Total Current Liabilities'])*100

Dates
2022    1.340766
2021    0.759763
2020    0.219465
2019    0.352823
2018    0.871215
dtype: float64

In [29]:
#avg yield on debt

avg_yield_on_debt=((df1['Interest Paid']/df1['Total Current Liabilities'])*100).mean()
print(avg_yield_on_debt)

0.7088064240883105


In [30]:
tax_schield=((df['Income Tax']/df['Pretax Income'])*100).mean()
print(tax_schield)

16.741453771698303


In [31]:
#cost of debt
cod=((avg_yield_on_debt/100)*(1-(tax_schield/100)))*100
print(cod)

0.5901419242687382


# **The calculated cost of debt, 0.5901 (or approximately 0.59), represents the effective interest rate that the company pays on its debt obligations. It indicates the cost incurred by the company to borrow funds through debt instruments. This percentage value can be used as a reference for evaluating the company's debt-related expenses and determining the overall cost of financing through debt.**

# Calculating CAPM

In [32]:
# % of equity
total_equity=6298  #2022
total_liabilities=14171  #2022
per_equity=(6298/(14171+6298))*100 
print(per_equity)

30.76847916361327


In [33]:
# % of debt
per_debt=(14171/(14171+6298))*100
print(per_debt)

69.23152083638672


In [34]:
#weighted average cost
((coe*per_equity)+(cod*per_debt))/100

9.554913042731734

NPV

cashflowschedule

Net Operating Cash Flow Growth	23.84%	34.62%	-12.61%	26.71%

Net Investing Cash Flow Growth	72.73%	-180.34%	-17.56%	-224.11%

Net Financing Cash Flow Growth	-60.76%	-205.37%	64.16%	-18.14%	

Free Cash Flow Growth	24.54%	35.37%	-12.21%	26.65%	

Net Change in Cash	401	2,694	3,124	342	751

In [35]:
#cash flow growth
import pandas as pd

data3= {
    'Year': ['2022', '2021', '2020', '2019', '2018'],
    'Net Change in Cash': [401, 2694, 3124, 342, 751]
}

df3= pd.DataFrame(data3)
df3['Cash Flow Growth'] = df3['Net Change in Cash'].pct_change()
df3['Cash Flow Growth'] = df3['Cash Flow Growth'].shift(-1)



In [36]:
df3.head()

,Year,Net Change in Cash,Cash Flow Growth
0,2022,401,5.718204
1,2021,2694,0.159614
2,2020,3124,-0.890525
3,2019,342,1.195906
4,2018,751,NaN


Cash flow Schedule with discounted rate calculation